## 1. データをクローリングする

In [98]:
! mkdir -p html excel csv tokenized vector log

In [ ]:
# 事故事例(Excel系)と、事例・状況・原因・対策（HTML系）の2種類を取得する。
! ./get_doc.sh > ./log/get_doc.log 2>&1 

## 2. 取得したデータをCSVに変換する。
* 事故事例のExcelと、事例・状況・原因・対策のHTMLをcsvに変換する。

In [100]:
# Excel to CSV
!python excel_to_csv.py ./excel/ ./csv/ > ./log/excel_to_csv.log 2>&1

In [99]:
# HTML to CSV
!python html_to_csv.py ./html/ ./csv/anzen_all.csv > ./log/html_to_csv.log 2>&1 

## 3. ElasticsearchにIndexを登録する

In [103]:
# 事例・状況・原因・対策用のインデックスを登録
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/anzen -d @es_anzen_csv_schema.txt

{"acknowledged":true,"shards_acknowledged":true,"index":"anzen"}

In [101]:
# 事例用のインデックスを登録
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/accident -d @es_accident_schema.txt

{"acknowledged":true,"shards_acknowledged":true,"index":"accident"}

## 4. Elasticsearchにデータをロードする

In [104]:
# 事例・状況・原因・対策データをロード(anzenインデックス)
! python load_anzen_es.py --host elasticsearch --index anzen --input_csv ./csv/anzen_all.csv  > ./log/load_anzen_es.log 2>&1

In [105]:
# 事例用データをロード(accidentインデックス)
!python load_accident_es.py --host elasticsearch --index accident --input_dir ./csv/  > ./log/load_accident_es.log 2>&1

## 5. Elasticsearchでトークナイズし、結果をCSV形式とテキスト形式で保存する

In [108]:
# 事例・状況・原因・対策データの文章をトークナイズし、CSVとTXTファイルとして保存する
!python es_anzen_tokenize.py --host elasticsearch --index anzen --output tokenized/anzen_tokenized > ./log/es_anzen_tokenize.log 2>&1

In [109]:
# 事例用データの文章をトークナイズし、CSVとTXTファイルとして保存する
!python es_accident_tokenize.py --host elasticsearch --index accident --output tokenized/accident_tokenized

## 6. GloVeで単語を学習する

In [106]:
# GloVeを取得する。最初に1回だけ実行すれば良い。
! git clone https://github.com/stanfordnlp/GloVe.git
! cd GloVe && make

fatal: destination path 'GloVe' already exists and is not an empty directory.
mkdir -p build


In [110]:
# トークナイズしたanzenとaccidentのテキストファイルを結合する。
! cat tokenized/anzen_tokenized.txt tokenized/accident_tokenized.txt > tokenized/all_tokens.txt

In [111]:
# GloVeで学習する。
! ./glove.sh 


$ GloVe/build/vocab_count -min-count 0 -verbose 2 < tokenized/all_tokens.txt > ./vector/glove_vocab.txt
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.Processed 1120989 tokens.
Counted 22222 unique words.
Using vocabulary of size 22222.

$ GloVe/build/cooccur -memory 4.0 -vocab-file ./vector/glove_vocab.txt -verbose 2 -window-size 15 < tokenized/all_tokens.txt > ./vector/glove_cooccurrence.bin
COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "./vector/glove_vocab.txt"...loaded 22222 words.
Building lookup table...table contains 62977881 elements.
Processing token: 010000020000030000040000050000060000070000080000090000010000001100000Processed 1120989 tokens.
Writing cooccurrences to disk........2 files in total.
Merging cooccurrence files: processed 0 lines.100

# SCDVで文章をベクトル化する

In [112]:
# anzenとaccidentのCSVをマージする。ベクトル化に必要な列のみ選別し、列名を統一する。
!python merge_csv.py --input_anzen tokenized/anzen_tokenized.csv --input_accident tokenized/accident_tokenized.csv --output_csv tokenized/merge_tokenized.csv

# LDAトピックモデルを使用して、いくつのトピックに分類すると良いか確認する
* 以降のSCDVモデルを構築する際の、GMMのクラスタ数の参考とするため
* 調べたクラスタ数 × 50(GloVeのベクトルサイズ)がSCDVのベクトルサイズとなる。このサイズでElasticsearch用のベクトルインデックスを登録する。
* 以下でElasticsearch用のベクトルインデックスを登録するが、es_vector_schema.txx内の"dense_vector"の"dimes"をこのベクトルサイズに書き換えること。

In [113]:
# ベクターインデックスを登録
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/vector -d @es_vector_schema.txt

{"acknowledged":true,"shards_acknowledged":true,"index":"vector"}

## SCDVのGMMモデルを構築する。
* 引数のnum_clustersは上記で調べたLDAトピックモデル数を参考にして設定する。

In [115]:
!python scdv.py --csv_file ./tokenized/merge_tokenized.csv --num_clusters 4

Clustering Done...
Cluster Assignments Saved...
Probabilities of Cluster Assignments saved...
100%|████████████████████████████████████| 4683/4683 [00:00<00:00, 23086.18it/s]
train size:42143  vector size:200
test size:4683  vector size:200
Test start...
Report
              precision    recall  f1-score   support

       cause   0.497126  0.394977  0.440204       438
    measures   0.646934  0.612000  0.628983       500
   situation   0.546419  0.422998  0.476852       487
       title   0.400000  0.160920  0.229508        87
      その他の事業   0.358209  0.131868  0.192771       182
      保健・衛生業   0.000000  0.000000  0.000000         1
        保健衛生   0.000000  0.000000  0.000000         1
       保健衛生業   0.000000  0.000000  0.000000        16
          商業   0.439252  0.320819  0.370809       293
         官公署   0.000000  0.000000  0.000000         1
         建設業   0.566540  0.749371  0.645254      1193
       接客娯楽業   0.600000  0.062500  0.113208        48
      教育・研究業   0.000000  0.000000  

In [117]:
# 全データについてSCDVベクトルを算出し、Elasticsearchに登録する
!python scdv_to_es.py --host elasticsearch --input_csv ./tokenized/merge_tokenized.csv --num_clusters 4 > ./log/scdv_to_es.log 2>&1 